In [ ]:
import pandas as pd
import re

# Đường dẫn đến 3 file CSV
file_paths = [
    'tpc-h/20gb/run1/query_metrics_worst_case_run1.csv',
    'tpc-h/20gb/run2/query_metrics_worst_case_run2.csv',
    'tpc-h/20gb/run3/query_metrics_worst_case_run3.csv'
]

# Các cột cần thiết
columns_to_use = [
    'query_id',
    'time (ms)',
    'avg_cpu_usage',
    'avg_gpu_usage',
    'avg_ram_usage',
    'avg_vram_usage',
    'avg_power_usage'
]

# Đọc, lọc, và làm sạch dữ liệu
dfs = [pd.read_csv(path, comment='/')[columns_to_use].assign(query_id=lambda df: df['query_id'].str.strip()) for path in file_paths]

# Gộp dữ liệu và tính trung bình theo query_id
all_data = pd.concat(dfs)
average_metrics = all_data.groupby('query_id', as_index=False).mean(numeric_only=True)
average_metrics = average_metrics.round(2)
# Thêm cột số thứ tự để sort theo số trong query_id
average_metrics['query_number'] = average_metrics['query_id'].apply(lambda x: int(re.search(r'\d+', x).group()))

# Sắp xếp theo số thứ tự
average_metrics = average_metrics.sort_values(by='query_number').drop(columns=['query_number'])

# Ghi ra file
output_path = 'average_query_metrics_worst_case_20gb.csv'
average_metrics.to_csv(output_path, index=False)
